# 

Credits:

The following code is my take on the tutorial from [fast.ai](https://github.com/fastai/courses/blob/master/deeplearning2/neural-style.ipynb) and the pytorch implementation from [sunshineatnoon](https://github.com/sunshineatnoon/Paper-Implementations/tree/master/NeuralSytleTransfer). This follows closely the articles [Neural Style Transfer]() and 

For the images the links are [van gogh self portrait](), [starry night]() and [pixelart](https://www.pinterest.fr/pin/726979564822222007/)

All links where functional at 16/04/2018

In [7]:
%matplotlib inline
import keras
from keras import backend as K
import scipy
import skimage
import skimage.transform
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize
import imageio
import tqdm
import tensorflow as tf

Now we choose the shape for our images, and set the paths for the different images we are going to use 

In [2]:
shape = (400,400,3)
content_filename = "images/me.jpg"
vangogh_filename = "images/VanGogh.jpg"
retro_filename = "images/pixelart.jpeg"

We create our network from the weights available in keras. Here I use VGG16 as it is the most used for this application. I also tried using Resnet50 and InceptionV2 but the results were not as good.

In the end we print the layers so that we can choose which ones we are going to use. The original paper suggests that using layers at the start for 

In [3]:
network_class = keras.applications.vgg16
model = network_class.VGG16(include_top=False,input_shape=shape)

print("Model layers:")
for a in model.layers:
    print(a.name)

Model layers:
input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [4]:
style_layers = [model.get_layer('block{}_conv1'.format(o)).output for o in range(1,6)]
content_layers = [model.get_layer('block{}_conv1'.format(o)).output for o in range(1,6)]


In [5]:
preprocess = network_class.preprocess_input

mean = np.array([123.68, 116.779, 103.939], dtype=np.float32) #Extracted from keras.applications.imagenet_utils

def deprocess(processed_image):
    return np.clip(processed_image[:, :, ::-1] + mean, 0, 255)/255

def load_image(filename,shape):
    image = plt.imread(filename) 
    if image.shape[2] == 4:
        image = image[:,:,:3]
    resized = skimage.transform.resize(image,shape,mode="constant").astype(np.float32) * 255 
    return resized,preprocess(resized.copy())


In [6]:
content,content_preprocessed = load_image(content_filename,shape)
vangogh,vangogh_preprocessed = load_image(vangogh_filename,shape)
pixelated,pixelated_preprocessed = load_image(pixelated_filename,shape)

fig,ax = plt.subplots(1,3,figsize=(20,10))
ax[0].imshow(content/255)
ax[1].imshow(vangogh/255)                      
ax[2].imshow(pixelated/255)                      

fig,ax = plt.subplots(1,3,figsize=(20,10))
ax[0].imshow(deprocess(content_preprocessed))
ax[1].imshow(deprocess(vangogh_night_preprocessed))                      
ax[2].imshow(deprocess(pixelated_preprocessed))                      


NameError: name 'pixelated_filename' is not defined

In [ ]:
class Evaluator(object):
    def __init__(self, f, shp): self.f, self.shp = f, shp
        
    def loss(self, x):
        loss_, self.grad_values = self.f([x.reshape(self.shp)])
        return loss_.astype(np.float64)

    def grads(self, x): return self.grad_values.flatten().astype(np.float64)
    
def mse(layer,target):
    return K.mean((layer-target)**2)

def solve_image(eval_obj, niter, x,name,iq):
    for i in tqdm.tqdm(list(range(niter))):
        x, min_val, info = scipy.optimize.fmin_l_bfgs_b(eval_obj.loss, x.flatten(),
                                         fprime=eval_obj.grads, maxfun=20)
        x = np.clip(x, -127,127)
        print('Current loss value:', min_val)
        imageio.imwrite("results/{}_at_iteration_{}.png".format(name,i), 
                        (deprocess(join_yiq_to_bgr(x.reshape(shape),iq))*255).astype(np.uint8))
    return x

def gram_matrix(x):
    # We want each row to be a channel, and the columns to be flattened x,y locations
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # The dot product of this with its transpose shows the correlation 
    # between each pair of channels
    the_dot = K.dot(features, K.transpose(features))
    tensor = tf.cast(K.prod(x.get_shape()[1:3]),tf.float32)
    num_elems = tensor
    return the_dot / num_elems

def style_loss(x, targ): 
    return mse(gram_matrix(x), gram_matrix(targ))

def style_transfer(input_content,input_style,content_multiplier,name,iq): 
    input_content = np.expand_dims(input_content,0)
    input_style = np.expand_dims(input_style,0)
    
    content_model = keras.Model(model.input, content_layers)
    style_model = keras.Model(model.input, style_layers)

    content_targets = [K.variable(output) for output in content_model.predict(input_content)]
    style_targets = [K.variable(output) for output in style_model.predict(input_style)]    
    
    style_losses = [style_loss(layer[0], target[0]) for layer,target in zip(style_layers, style_targets)]
    content_losses = [mse(layer, target) for layer,target in zip(content_layers, content_targets)]

    _style_loss = K.mean(K.stack(style_losses))
    content_loss = K.mean(K.stack(content_losses))
    
    loss = content_multiplier*content_loss + _style_loss
    
    grads = K.gradients(loss, model.input)
    fn = K.function([model.input], [loss]+grads)
    evaluator = Evaluator(fn, input_content.shape)
    
    x = input_content.copy()#np.random.uniform(-2.5, 2.5, [1]+list(shape))/100
    x = solve_image(evaluator, 5, x,name,iq)
    x = x.reshape(shape)

    fig,ax = plt.subplots(1,3,figsize=(20,10))
    ax[0].imshow(deprocess(join_yiq_to_bgr(input_content[0],iq)))
    ax[1].imshow(deprocess(join_yiq_to_bgr(input_style[0],iq)))                      
    ax[2].imshow(deprocess(join_yiq_to_bgr(x,iq)))       
    
    del evaluator, fn, grads, loss, content_targets,style_targets,content_model,style_model
    return x

In [ ]:
def bgr_to_yiq(x):
    transform = np.asarray([[0.114, 0.587, 0.299], [-0.322, -0.274, 0.596], [0.312, -0.523, 0.211]], dtype=np.float32)
    h, w, c = x.shape
    x = x.transpose((2, 0, 1)).reshape((c, -1))
    x = transform.dot(x)
    return x.reshape((c, h, w)).transpose((1, 2, 0))

def yiq_to_bgr(x):
    transform = np.asarray([[1, -1.106, 1.703], [1, -0.272, -0.647], [1, 0.956, 0.621]], dtype=np.float32)
    h, w, c = x.shape
    x = x.transpose((2, 0, 1)).reshape((c, -1))
    x = transform.dot(x)
    return x.reshape((c, h, w)).transpose((1, 2, 0))

def split_bgr_to_yiq(x):
    x = bgr_to_yiq(x)
    y = x[:,:,0:1]
    iq = x[:,:,1:]
    return np.repeat(y, 3, axis=2), iq

def join_yiq_to_bgr(y, iq):
    y = bgr_to_yiq(y)[:,:,0:1]
    return yiq_to_bgr(np.concatenate((y, iq), axis=2))

def luminance_transfer(x,y):
    # x: style, y:content
    x_l, x_iq = split_bgr_to_yiq(x) # 1x3x512x512
    y_l, y_iq = split_bgr_to_yiq(y)

    x_l_mean = np.mean(x_l)
    y_l_mean = np.mean(y_l)
    x_l_std = np.std(x_l)
    y_l_std = np.std(y_l)

    x_l = (y_l_std/x_l_std)*(x_l - x_l_mean) + y_l_mean
    return x_l, y_l, y_iq


In [ ]:
vangogh_luminance, content_luminance_vangogh, vangogh_iq = luminance_transfer(vangogh_preprocessed,content_preprocessed)


me_vangogh = style_transfer(content_luminance_vangogh,vangogh_luminance,1,"starry_night",vangogh_iq)

In [ ]:
outImg = join_yiq_to_bgr(me_vangogh,vangogh_iq)
deprocessed = deprocess(outImg)
plt.figure(figsize=(20,20))
plt.imshow(deprocessed)
deprocessed.min(),deprocessed.max()
imageio.imwrite("results/me_gogh.png",(deprocessed*255).astype(np.uint8))

In [ ]:
pixelated_luminance, content_luminance_pixelated, pixelated_iq = luminance_transfer(pixelated_preprocessed,content_preprocessed)


me_pixelated = style_transfer(content_luminance_pixelated,pixelated_luminance,100,"me_pixel",pixelated_iq)